In [2]:
# https://julien.danjou.info/blog/2013/guide-python-static-class-abstract-methods

<p>Doing code reviews is a great way to discover things that people might
struggle to comprehend. While proof-reading
<a href="http://review.openstack.org">OpenStack patches</a> recently, I spotted that
people were not using correctly the various decorators Python provides for
methods. So here's my attempt at providing me a link to send them to in my
next code reviews. :-)</p>
<p>This article has been kindly converted and recorded as a video by
<a href="https://www.webucator.com/programming/python.cfm">Webucator, provider of Python training</a>.
It's a good alternative to reading, if you prefer listening to someone and
following step-by-step explanation of the code..</p>

<h1>How methods work in Python</h1>
<p>A method is a function that is stored as a class attribute. You can declare
and access such a function this way:</p>
<div class="highlight"><pre><span></span><span class="o">&gt;&gt;&gt;</span> <span class="k">class</span> <span class="nc">Pizza</span><span class="p">(</span><span class="nb">object</span><span class="p">):</span><br /><span class="o">...</span>     <span class="k">def</span> <span class="fm">__init__</span><span class="p">(</span><span class="bp">self</span><span class="p">,</span> <span class="n">size</span><span class="p">):</span><br /><span class="o">...</span>         <span class="bp">self</span><span class="o">.</span><span class="n">size</span> <span class="o">=</span> <span class="n">size</span><br /><span class="o">...</span>     <span class="k">def</span> <span class="nf">get_size</span><span class="p">(</span><span class="bp">self</span><span class="p">):</span><br /><span class="o">...</span>         <span class="k">return</span> <span class="bp">self</span><span class="o">.</span><span class="n">size</span><br /><span class="o">...</span><br /><span class="o">&gt;&gt;&gt;</span> <span class="n">Pizza</span><span class="o">.</span><span class="n">get_size</span><br /><span class="o">&lt;</span><span class="n">unbound</span> <span class="n">method</span> <span class="n">Pizza</span><span class="o">.</span><span class="n">get_size</span><span class="o">&gt;</span><br /></pre></div>

<p><br />
What Python tells you here, is that the attribute <em>get_size</em> of the class
<em>Pizza</em> is a method that is <strong>unbound</strong>. What does this mean? We'll know as
soon as we'll try to call it:</p>
<div class="highlight"><pre><span></span><span class="o">&gt;&gt;&gt;</span> <span class="n">Pizza</span><span class="o">.</span><span class="n">get_size</span><span class="p">()</span><br /><span class="n">Traceback</span> <span class="p">(</span><span class="n">most</span> <span class="n">recent</span> <span class="n">call</span> <span class="n">last</span><span class="p">):</span><br />  <span class="n">File</span> <span class="s2">&quot;&lt;stdin&gt;&quot;</span><span class="p">,</span> <span class="n">line</span> <span class="mi">1</span><span class="p">,</span> <span class="ow">in</span> <span class="o">&lt;</span><span class="n">module</span><span class="o">&gt;</span><br /><span class="ne">TypeError</span><span class="p">:</span> <span class="n">unbound</span> <span class="n">method</span> <span class="n">get_size</span><span class="p">()</span> <span class="n">must</span> <span class="n">be</span> <span class="n">called</span> <span class="k">with</span> <span class="n">Pizza</span> <span class="n">instance</span> <span class="k">as</span> <span class="n">first</span> <span class="n">argument</span> <span class="p">(</span><span class="n">got</span> <span class="n">nothing</span> <span class="n">instead</span><span class="p">)</span><br /></pre></div>

<p><br />
We can't call it because it's not bound to any instance of <em>Pizza</em>. And a
method wants an instance as its first argument (in Python 2 it <strong>must</strong> be
an instance of that class; in Python 3 it could be anything). Let's try to
do that then:</p>
<div class="highlight"><pre><span></span><span class="o">&gt;&gt;&gt;</span> <span class="n">Pizza</span><span class="o">.</span><span class="n">get_size</span><span class="p">(</span><span class="n">Pizza</span><span class="p">(</span><span class="mi">42</span><span class="p">))</span><br /><span class="mi">42</span><br /></pre></div>

<p><br />
It worked! We called the method with an instance as its first argument, so
everything's fine. But you will agree with me if I say this is not a very
handy way to call methods; we have to refer to the class each time we want
to call a method. And if we don't know what class is our object, this is not
going to work for very long.</p>
<p>So what Python does for us, is that it binds all the methods from the class
<em>Pizza</em> to any instance of this class. This means that the attribute
<em>get_size</em> of an instance of <em>Pizza</em> is a bound method: a method for
which the first argument will be the instance itself.</p>
<div class="highlight"><pre><span></span><span class="o">&gt;&gt;&gt;</span> <span class="n">Pizza</span><span class="p">(</span><span class="mi">42</span><span class="p">)</span><span class="o">.</span><span class="n">get_size</span><br /><span class="o">&lt;</span><span class="n">bound</span> <span class="n">method</span> <span class="n">Pizza</span><span class="o">.</span><span class="n">get_size</span> <span class="n">of</span> <span class="o">&lt;</span><span class="n">__main__</span><span class="o">.</span><span class="n">Pizza</span> <span class="nb">object</span> <span class="n">at</span> <span class="mh">0x7f3138827910</span><span class="o">&gt;&gt;</span><br /><span class="o">&gt;&gt;&gt;</span> <span class="n">Pizza</span><span class="p">(</span><span class="mi">42</span><span class="p">)</span><span class="o">.</span><span class="n">get_size</span><span class="p">()</span><br /><span class="mi">42</span><br /></pre></div>

<p><br />
As expected, we don't have to provide any argument to <em>get_size</em>, since
it's bound, its <em>self</em> argument is automatically set to our <em>Pizza</em>
instance. Here's an even better proof of that:</p>
<div class="highlight"><pre><span></span><span class="o">&gt;&gt;&gt;</span> <span class="n">m</span> <span class="o">=</span> <span class="n">Pizza</span><span class="p">(</span><span class="mi">42</span><span class="p">)</span><span class="o">.</span><span class="n">get_size</span><br /><span class="o">&gt;&gt;&gt;</span> <span class="n">m</span><span class="p">()</span><br /><span class="mi">42</span><br /></pre></div>

<p><br />
Indeed, you don't even have to keep a reference to your <em>Pizza</em> object. Its
method is bound to the object, so the method is sufficient to itself.</p>
<p>But what if you wanted to know which object this bound method is bound to?
Here's a little trick:</p>
<div class="highlight"><pre><span></span><span class="o">&gt;&gt;&gt;</span> <span class="n">m</span> <span class="o">=</span> <span class="n">Pizza</span><span class="p">(</span><span class="mi">42</span><span class="p">)</span><span class="o">.</span><span class="n">get_size</span><br /><span class="o">&gt;&gt;&gt;</span> <span class="n">m</span><span class="o">.</span><span class="vm">__self__</span><br /><span class="o">&lt;</span><span class="n">__main__</span><span class="o">.</span><span class="n">Pizza</span> <span class="nb">object</span> <span class="n">at</span> <span class="mh">0x7f3138827910</span><span class="o">&gt;</span><br /><span class="o">&gt;&gt;&gt;</span> <span class="c1"># You could guess, look at this:</span><br /><span class="o">...</span><br /><span class="o">&gt;&gt;&gt;</span> <span class="n">m</span> <span class="o">==</span> <span class="n">m</span><span class="o">.</span><span class="vm">__self__</span><span class="o">.</span><span class="n">get_size</span><br /><span class="bp">True</span><br /></pre></div>

<p><br />
Obviously, we still have a reference to our object, and we can find it back
if we want.</p>
<p>In Python 3, the functions attached to a class are not considered as
<em>unbound method</em> anymore, but as simple functions, that are bound to an
object if required. So the principle stays the same, the model is just
simplified.</p>
<div class="highlight"><pre><span></span><span class="o">&gt;&gt;&gt;</span> <span class="k">class</span> <span class="nc">Pizza</span><span class="p">(</span><span class="nb">object</span><span class="p">):</span><br /><span class="o">...</span>     <span class="k">def</span> <span class="fm">__init__</span><span class="p">(</span><span class="bp">self</span><span class="p">,</span> <span class="n">size</span><span class="p">):</span><br /><span class="o">...</span>         <span class="bp">self</span><span class="o">.</span><span class="n">size</span> <span class="o">=</span> <span class="n">size</span><br /><span class="o">...</span>     <span class="k">def</span> <span class="nf">get_size</span><span class="p">(</span><span class="bp">self</span><span class="p">):</span><br /><span class="o">...</span>         <span class="k">return</span> <span class="bp">self</span><span class="o">.</span><span class="n">size</span><br /><span class="o">...</span><br /><span class="o">&gt;&gt;&gt;</span> <span class="n">Pizza</span><span class="o">.</span><span class="n">get_size</span><br /><span class="o">&lt;</span><span class="n">function</span> <span class="n">Pizza</span><span class="o">.</span><span class="n">get_size</span> <span class="n">at</span> <span class="mh">0x7f307f984dd0</span><span class="o">&gt;</span><br /></pre></div>

<p><br /></p>
<h1>Static methods</h1>
<p>Static methods are a special case of methods. Sometimes, you'll write code
that belongs to a class, but that doesn't use the object itself at all. For
example:</p>
<div class="highlight"><pre><span></span><span class="k">class</span> <span class="nc">Pizza</span><span class="p">(</span><span class="nb">object</span><span class="p">):</span><br />    <span class="nd">@staticmethod</span><br />    <span class="k">def</span> <span class="nf">mix_ingredients</span><span class="p">(</span><span class="n">x</span><span class="p">,</span> <span class="n">y</span><span class="p">):</span><br />        <span class="k">return</span> <span class="n">x</span> <span class="o">+</span> <span class="n">y</span><br />&nbsp;<br />    <span class="k">def</span> <span class="nf">cook</span><span class="p">(</span><span class="bp">self</span><span class="p">):</span><br />        <span class="k">return</span> <span class="bp">self</span><span class="o">.</span><span class="n">mix_ingredients</span><span class="p">(</span><span class="bp">self</span><span class="o">.</span><span class="n">cheese</span><span class="p">,</span> <span class="bp">self</span><span class="o">.</span><span class="n">vegetables</span><span class="p">)</span><br /></pre></div>

<p><br />
In such a case, writing <em>mix_ingredients</em> as a non-static method would work
too, but it would provide it with a <em>self</em> argument that would not be used. Here,
the decorator <em>@staticmethod</em> buys us several things:</p>
<ul>
<li>Python doesn't have to instantiate a bound-method for each <em>Pizza</em> object
we instantiate. Bound methods are objects too, and creating them has a
cost. Having a static method avoids that:</li>
</ul>
<div class="highlight"><pre><span></span><span class="o">&gt;&gt;&gt;</span> <span class="n">Pizza</span><span class="p">()</span><span class="o">.</span><span class="n">cook</span> <span class="ow">is</span> <span class="n">Pizza</span><span class="p">()</span><span class="o">.</span><span class="n">cook</span><br /><span class="bp">False</span><br /><span class="o">&gt;&gt;&gt;</span> <span class="n">Pizza</span><span class="p">()</span><span class="o">.</span><span class="n">mix_ingredients</span> <span class="ow">is</span> <span class="n">Pizza</span><span class="o">.</span><span class="n">mix_ingredients</span><br /><span class="bp">True</span><br /><span class="o">&gt;&gt;&gt;</span> <span class="n">Pizza</span><span class="p">()</span><span class="o">.</span><span class="n">mix_ingredients</span> <span class="ow">is</span> <span class="n">Pizza</span><span class="p">()</span><span class="o">.</span><span class="n">mix_ingredients</span><br /><span class="bp">True</span><br /></pre></div>

<p><br /></p>
<ul>
<li>
<p>It eases the readability of the code: seeing <em>@staticmethod</em>, we know that
the method does not depend on the state of the object itself;</p>
</li>
<li>
<p>It allows us to override the <em>mix_ingredients</em> method in a subclass. If we
used a function <em>mix_ingredients</em> defined at the top-level of our module, a
class inheriting from <em>Pizza</em> wouldn't be able to change the way we mix
ingredients for our pizza without overriding <em>cook</em> itself.</p>
</li>
</ul>
<h1>Class methods</h1>
<p>Having said that, what are class methods? Class methods are methods that are
not bound to an object, but to… a class!</p>
<div class="highlight"><pre><span></span><span class="o">&gt;&gt;&gt;</span> <span class="k">class</span> <span class="nc">Pizza</span><span class="p">(</span><span class="nb">object</span><span class="p">):</span><br /><span class="o">...</span>     <span class="n">radius</span> <span class="o">=</span> <span class="mi">42</span><br /><span class="o">...</span>     <span class="nd">@classmethod</span><br /><span class="o">...</span>     <span class="k">def</span> <span class="nf">get_radius</span><span class="p">(</span><span class="bp">cls</span><span class="p">):</span><br /><span class="o">...</span>         <span class="k">return</span> <span class="bp">cls</span><span class="o">.</span><span class="n">radius</span><br /><span class="o">...</span> <br /><span class="o">&gt;&gt;&gt;</span> <br /><span class="o">&gt;&gt;&gt;</span> <span class="n">Pizza</span><span class="o">.</span><span class="n">get_radius</span><br /><span class="o">&lt;</span><span class="n">bound</span> <span class="n">method</span> <span class="nb">type</span><span class="o">.</span><span class="n">get_radius</span> <span class="n">of</span> <span class="o">&lt;</span><span class="k">class</span> <span class="err">&#39;</span><span class="nc">__main__</span><span class="o">.</span><span class="n">Pizza</span><span class="s1">&#39;&gt;&gt;</span><br /><span class="o">&gt;&gt;&gt;</span> <span class="n">Pizza</span><span class="p">()</span><span class="o">.</span><span class="n">get_radius</span><br /><span class="o">&lt;</span><span class="n">bound</span> <span class="n">method</span> <span class="nb">type</span><span class="o">.</span><span class="n">get_radius</span> <span class="n">of</span> <span class="o">&lt;</span><span class="k">class</span> <span class="err">&#39;</span><span class="nc">__main__</span><span class="o">.</span><span class="n">Pizza</span><span class="s1">&#39;&gt;&gt;</span><br /><span class="o">&gt;&gt;&gt;</span> <span class="n">Pizza</span><span class="o">.</span><span class="n">get_radius</span> <span class="ow">is</span> <span class="n">Pizza</span><span class="p">()</span><span class="o">.</span><span class="n">get_radius</span><br /><span class="bp">True</span><br /><span class="o">&gt;&gt;&gt;</span> <span class="n">Pizza</span><span class="o">.</span><span class="n">get_radius</span><span class="p">()</span><br /><span class="mi">42</span><br /></pre></div>

<p><br />
Whatever the way you use to access this method, it will always be bound to the
class it is attached to, and its first argument will be the class itself
(remember that classes are objects too).</p>
<p>When to use this kind of methods? Well class methods are mostly useful for
two types of methods:</p>
<ul>
<li>Factory methods, that are used to create an instance for a class using for
example some sort of pre-processing. If we use a <em>@staticmethod</em> instead, we
would have to hardcode the <em>Pizza</em> class name in our function, making any
class inheriting from <em>Pizza</em> unable to use our factory for its own use.</li>
</ul>
<div class="highlight"><pre><span></span><span class="k">class</span> <span class="nc">Pizza</span><span class="p">(</span><span class="nb">object</span><span class="p">):</span><br />    <span class="k">def</span> <span class="fm">__init__</span><span class="p">(</span><span class="bp">self</span><span class="p">,</span> <span class="n">ingredients</span><span class="p">):</span><br />        <span class="bp">self</span><span class="o">.</span><span class="n">ingredients</span> <span class="o">=</span> <span class="n">ingredients</span><br />&nbsp;<br />    <span class="nd">@classmethod</span><br />    <span class="k">def</span> <span class="nf">from_fridge</span><span class="p">(</span><span class="bp">cls</span><span class="p">,</span> <span class="n">fridge</span><span class="p">):</span><br />        <span class="k">return</span> <span class="bp">cls</span><span class="p">(</span><span class="n">fridge</span><span class="o">.</span><span class="n">get_cheese</span><span class="p">()</span> <span class="o">+</span> <span class="n">fridge</span><span class="o">.</span><span class="n">get_vegetables</span><span class="p">())</span><br /></pre></div>

<p><br /></p>
<ul>
<li>Static methods calling static methods: if you split a static method in
several static methods, you shouldn't hard-code the class name but use class
methods. Using this way to declare our method, the <em>Pizza</em> name is never
directly referenced and inheritance and method overriding will work flawlessly</li>
</ul>
<div class="highlight"><pre><span></span><span class="k">class</span> <span class="nc">Pizza</span><span class="p">(</span><span class="nb">object</span><span class="p">):</span><br />    <span class="k">def</span> <span class="fm">__init__</span><span class="p">(</span><span class="bp">self</span><span class="p">,</span> <span class="n">radius</span><span class="p">,</span> <span class="n">height</span><span class="p">):</span><br />        <span class="bp">self</span><span class="o">.</span><span class="n">radius</span> <span class="o">=</span> <span class="n">radius</span><br />        <span class="bp">self</span><span class="o">.</span><span class="n">height</span> <span class="o">=</span> <span class="n">height</span><br />&nbsp;<br />    <span class="nd">@staticmethod</span><br />    <span class="k">def</span> <span class="nf">compute_area</span><span class="p">(</span><span class="n">radius</span><span class="p">):</span><br />         <span class="k">return</span> <span class="n">math</span><span class="o">.</span><span class="n">pi</span> <span class="o">*</span> <span class="p">(</span><span class="n">radius</span> <span class="o">**</span> <span class="mi">2</span><span class="p">)</span><br />&nbsp;<br />    <span class="nd">@classmethod</span><br />    <span class="k">def</span> <span class="nf">compute_volume</span><span class="p">(</span><span class="bp">cls</span><span class="p">,</span> <span class="n">height</span><span class="p">,</span> <span class="n">radius</span><span class="p">):</span><br />         <span class="k">return</span> <span class="n">height</span> <span class="o">*</span> <span class="bp">cls</span><span class="o">.</span><span class="n">compute_area</span><span class="p">(</span><span class="n">radius</span><span class="p">)</span><br />&nbsp;<br />    <span class="k">def</span> <span class="nf">get_volume</span><span class="p">(</span><span class="bp">self</span><span class="p">):</span><br />        <span class="k">return</span> <span class="bp">self</span><span class="o">.</span><span class="n">compute_volume</span><span class="p">(</span><span class="bp">self</span><span class="o">.</span><span class="n">height</span><span class="p">,</span> <span class="bp">self</span><span class="o">.</span><span class="n">radius</span><span class="p">)</span><br /></pre></div>

<p><br /></p>
<h1>Abstract methods</h1>
<p>An abstract method is a method defined in a base class, but that may not
provide any implementation. In Java, it would describe the methods of an
interface.</p>
<p>So the simplest way to write an abstract method in Python is:</p>
<div class="highlight"><pre><span></span><span class="k">class</span> <span class="nc">Pizza</span><span class="p">(</span><span class="nb">object</span><span class="p">):</span><br />    <span class="k">def</span> <span class="nf">get_radius</span><span class="p">(</span><span class="bp">self</span><span class="p">):</span><br />        <span class="k">raise</span> <span class="ne">NotImplementedError</span><br /></pre></div>

<p><br />
Any class inheriting from <em>Pizza</em> should implement and override the
<em>get_radius</em> method, otherwise an exception would be raised.</p>
<p>This particular way of implementing abstract method has a drawback. If you
write a class that inherits from <em>Pizza</em> and forget to implement
<em>get_radius</em>, the error will only be raised when you'll try to use that
method.</p>
<div class="highlight"><pre><span></span><span class="o">&gt;&gt;&gt;</span> <span class="n">Pizza</span><span class="p">()</span><br /><span class="o">&lt;</span><span class="n">__main__</span><span class="o">.</span><span class="n">Pizza</span> <span class="nb">object</span> <span class="n">at</span> <span class="mh">0x7fb747353d90</span><span class="o">&gt;</span><br /><span class="o">&gt;&gt;&gt;</span> <span class="n">Pizza</span><span class="p">()</span><span class="o">.</span><span class="n">get_radius</span><span class="p">()</span><br /><span class="n">Traceback</span> <span class="p">(</span><span class="n">most</span> <span class="n">recent</span> <span class="n">call</span> <span class="n">last</span><span class="p">):</span><br />  <span class="n">File</span> <span class="s2">&quot;&lt;stdin&gt;&quot;</span><span class="p">,</span> <span class="n">line</span> <span class="mi">1</span><span class="p">,</span> <span class="ow">in</span> <span class="o">&lt;</span><span class="n">module</span><span class="o">&gt;</span><br />  <span class="n">File</span> <span class="s2">&quot;&lt;stdin&gt;&quot;</span><span class="p">,</span> <span class="n">line</span> <span class="mi">3</span><span class="p">,</span> <span class="ow">in</span> <span class="n">get_radius</span><br /><span class="ne">NotImplementedError</span><br /></pre></div>

<p><br />
There's a way to trigger this way earlier, when the object is being
instantiated, using the <a href="http://docs.python.org/2/library/abc.html">abc</a>
module that's provided with Python.</p>
<div class="highlight"><pre><span></span><span class="kn">import</span> <span class="nn">abc</span><br />&nbsp;<br /><span class="k">class</span> <span class="nc">BasePizza</span><span class="p">(</span><span class="nb">object</span><span class="p">):</span><br />    <span class="vm">__metaclass__</span>  <span class="o">=</span> <span class="n">abc</span><span class="o">.</span><span class="n">ABCMeta</span><br />&nbsp;<br />    <span class="nd">@abc.abstractmethod</span><br />    <span class="k">def</span> <span class="nf">get_radius</span><span class="p">(</span><span class="bp">self</span><span class="p">):</span><br />         <span class="sd">&quot;&quot;&quot;Method that should do something.&quot;&quot;&quot;</span><br /></pre></div>

<p><br />
Using <em>abc</em> and its special class, as soon as you'll try to instantiate
<em>BasePizza</em> or any class inheriting from it, you'll get a <em>TypeError</em>.</p>
<div class="highlight"><pre><span></span><span class="o">&gt;&gt;&gt;</span> <span class="n">BasePizza</span><span class="p">()</span><br /><span class="n">Traceback</span> <span class="p">(</span><span class="n">most</span> <span class="n">recent</span> <span class="n">call</span> <span class="n">last</span><span class="p">):</span><br />  <span class="n">File</span> <span class="s2">&quot;&lt;stdin&gt;&quot;</span><span class="p">,</span> <span class="n">line</span> <span class="mi">1</span><span class="p">,</span> <span class="ow">in</span> <span class="o">&lt;</span><span class="n">module</span><span class="o">&gt;</span><br /><span class="ne">TypeError</span><span class="p">:</span> <span class="n">Can</span><span class="s1">&#39;t instantiate abstract class BasePizza with abstract methods get_radius</span><br /></pre></div>

<p><br /></p>
<h1>Mixing static, class and abstract methods</h1>
<p>When building classes and inheritances, the time will come where you will
have to mix all these methods decorators. So here's some tips about it.</p>
<p>Keep in mind that declaring a method as being abstract, doesn't freeze the
prototype of that method. That means that it must be implemented, but it can be
implemented with any argument list.</p>
<div class="highlight"><pre><span></span><span class="kn">import</span> <span class="nn">abc</span><br />&nbsp;<br /><span class="k">class</span> <span class="nc">BasePizza</span><span class="p">(</span><span class="nb">object</span><span class="p">):</span><br />    <span class="vm">__metaclass__</span>  <span class="o">=</span> <span class="n">abc</span><span class="o">.</span><span class="n">ABCMeta</span><br />&nbsp;<br />    <span class="nd">@abc.abstractmethod</span><br />    <span class="k">def</span> <span class="nf">get_ingredients</span><span class="p">(</span><span class="bp">self</span><span class="p">):</span><br />         <span class="sd">&quot;&quot;&quot;Returns the ingredient list.&quot;&quot;&quot;</span><br />&nbsp;<br /><span class="k">class</span> <span class="nc">Calzone</span><span class="p">(</span><span class="n">BasePizza</span><span class="p">):</span><br />    <span class="k">def</span> <span class="nf">get_ingredients</span><span class="p">(</span><span class="bp">self</span><span class="p">,</span> <span class="n">with_egg</span><span class="o">=</span><span class="bp">False</span><span class="p">):</span><br />        <span class="n">egg</span> <span class="o">=</span> <span class="n">Egg</span><span class="p">()</span> <span class="k">if</span> <span class="n">with_egg</span> <span class="k">else</span> <span class="bp">None</span><br />        <span class="k">return</span> <span class="bp">self</span><span class="o">.</span><span class="n">ingredients</span> <span class="o">+</span> <span class="n">egg</span><br /></pre></div>

<p><br />
This is valid, since <em>Calzone</em> fulfills the interface requirement we defined
for <em>BasePizza</em> objects. That means that we could also implement it as being
a class or a static method, for example:</p>
<div class="highlight"><pre><span></span><span class="kn">import</span> <span class="nn">abc</span><br />&nbsp;<br /><span class="k">class</span> <span class="nc">BasePizza</span><span class="p">(</span><span class="nb">object</span><span class="p">):</span><br />    <span class="vm">__metaclass__</span>  <span class="o">=</span> <span class="n">abc</span><span class="o">.</span><span class="n">ABCMeta</span><br />&nbsp;<br />    <span class="nd">@abc.abstractmethod</span><br />    <span class="k">def</span> <span class="nf">get_ingredients</span><span class="p">(</span><span class="bp">self</span><span class="p">):</span><br />         <span class="sd">&quot;&quot;&quot;Returns the ingredient list.&quot;&quot;&quot;</span><br />&nbsp;<br /><span class="k">class</span> <span class="nc">DietPizza</span><span class="p">(</span><span class="n">BasePizza</span><span class="p">):</span><br />    <span class="nd">@staticmethod</span><br />    <span class="k">def</span> <span class="nf">get_ingredients</span><span class="p">():</span><br />        <span class="k">return</span> <span class="bp">None</span><br /></pre></div>

<p><br />
This is also correct and fulfills the contract we have with our abstract
<em>BasePizza</em> class. The fact that the <em>get_ingredients</em> method doesn't need to know
about the object to return result is an implementation detail, not a
criteria to have our contract fulfilled.</p>
<p>Therefore, you can't force an implementation of your abstract method to be a
regular, class or static method, and arguably you shouldn't. Starting with
Python 3 (this won't work as you would expect in Python 2, see
<a href="http://bugs.python.org/issue5867">issue5867</a>), it's now possible to use the
<em>@staticmethod</em> and <em>@classmethod</em> decorators on top of <em>@abstractmethod</em>:</p>
<div class="highlight"><pre><span></span><span class="kn">import</span> <span class="nn">abc</span><br />&nbsp;<br /><span class="k">class</span> <span class="nc">BasePizza</span><span class="p">(</span><span class="nb">object</span><span class="p">):</span><br />    <span class="vm">__metaclass__</span>  <span class="o">=</span> <span class="n">abc</span><span class="o">.</span><span class="n">ABCMeta</span><br />&nbsp;<br />    <span class="n">ingredient</span> <span class="o">=</span> <span class="p">[</span><span class="s1">&#39;cheese&#39;</span><span class="p">]</span><br />&nbsp;<br />    <span class="nd">@classmethod</span><br />    <span class="nd">@abc.abstractmethod</span><br />    <span class="k">def</span> <span class="nf">get_ingredients</span><span class="p">(</span><span class="bp">cls</span><span class="p">):</span><br />         <span class="sd">&quot;&quot;&quot;Returns the ingredient list.&quot;&quot;&quot;</span><br />         <span class="k">return</span> <span class="bp">cls</span><span class="o">.</span><span class="n">ingredients</span><br /></pre></div>

<p><br />
Don't misread this: if you think this is going to force your subclasses to
implement <em>get_ingredients</em> as a class method, you are wrong. This simply
implies that your implementation of <em>get_ingredients</em> in the <em>BasePizza</em>
class is a class method.</p>
<p>An implementation in an abstract method? Yes! In Python, contrary to methods
in Java interfaces, you can have code in your abstract methods and call it
via <em>super()</em>:</p>
<div class="highlight"><pre><span></span><span class="kn">import</span> <span class="nn">abc</span><br />&nbsp;<br /><span class="k">class</span> <span class="nc">BasePizza</span><span class="p">(</span><span class="nb">object</span><span class="p">):</span><br />    <span class="vm">__metaclass__</span>  <span class="o">=</span> <span class="n">abc</span><span class="o">.</span><span class="n">ABCMeta</span><br />&nbsp;<br />    <span class="n">default_ingredients</span> <span class="o">=</span> <span class="p">[</span><span class="s1">&#39;cheese&#39;</span><span class="p">]</span><br />&nbsp;<br />    <span class="nd">@classmethod</span><br />    <span class="nd">@abc.abstractmethod</span><br />    <span class="k">def</span> <span class="nf">get_ingredients</span><span class="p">(</span><span class="bp">cls</span><span class="p">):</span><br />         <span class="sd">&quot;&quot;&quot;Returns the ingredient list.&quot;&quot;&quot;</span><br />         <span class="k">return</span> <span class="bp">cls</span><span class="o">.</span><span class="n">default_ingredients</span><br />&nbsp;<br /><span class="k">class</span> <span class="nc">DietPizza</span><span class="p">(</span><span class="n">BasePizza</span><span class="p">):</span><br />    <span class="k">def</span> <span class="nf">get_ingredients</span><span class="p">(</span><span class="bp">self</span><span class="p">):</span><br />        <span class="k">return</span> <span class="p">[</span><span class="s1">&#39;egg&#39;</span><span class="p">]</span> <span class="o">+</span> <span class="nb">super</span><span class="p">(</span><span class="n">DietPizza</span><span class="p">,</span> <span class="bp">self</span><span class="p">)</span><span class="o">.</span><span class="n">get_ingredients</span><span class="p">()</span><br /></pre></div>

<p><br />
In such a case, every pizza you will build by inheriting from <em>BasePizza</em>
will have to override the <em>get_ingredients</em> method, but will be able to use
the default mechanism to get the ingredient list by using <em>super()</em>.</p>  